In [1]:
import spikeinterface.full as si
import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from probeinterface.plotting import plot_probe
%matplotlib qt 

# Paths
base_folder = Path('/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/') 
folders = sorted(glob.glob((base_folder / 'Rec_*_g0').as_posix()))

In [2]:
folders

['/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_10_22_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_11_24_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_12_22_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_13_22_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_14_22_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_1_18_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_2_19_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_3_19_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_4_19_11_2021_g0',
 '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_20

In [3]:
folder= folders[-3]
print(folder)
output_folder = Path('/home/e4/Documents/temporary_rec6/')
#output_folder.mkdir(exist_ok=True, parents=True)

/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_6_22_11_2021_g0


In [4]:
rec_lf = si.SpikeGLXRecordingExtractor(folder, stream_id='imec0.lf') # LF
print(rec_lf)

SpikeGLXRecordingExtractor: 384 channels - 1 segments - 2.5kHz - 37446.090s


In [5]:
rec_lf.channel_ids

array(['imec0.lf#LF0', 'imec0.lf#LF1', 'imec0.lf#LF2', 'imec0.lf#LF3',
       'imec0.lf#LF4', 'imec0.lf#LF5', 'imec0.lf#LF6', 'imec0.lf#LF7',
       'imec0.lf#LF8', 'imec0.lf#LF9', 'imec0.lf#LF10', 'imec0.lf#LF11',
       'imec0.lf#LF12', 'imec0.lf#LF13', 'imec0.lf#LF14', 'imec0.lf#LF15',
       'imec0.lf#LF16', 'imec0.lf#LF17', 'imec0.lf#LF18', 'imec0.lf#LF19',
       'imec0.lf#LF20', 'imec0.lf#LF21', 'imec0.lf#LF22', 'imec0.lf#LF23',
       'imec0.lf#LF24', 'imec0.lf#LF25', 'imec0.lf#LF26', 'imec0.lf#LF27',
       'imec0.lf#LF28', 'imec0.lf#LF29', 'imec0.lf#LF30', 'imec0.lf#LF31',
       'imec0.lf#LF32', 'imec0.lf#LF33', 'imec0.lf#LF34', 'imec0.lf#LF35',
       'imec0.lf#LF36', 'imec0.lf#LF37', 'imec0.lf#LF38', 'imec0.lf#LF39',
       'imec0.lf#LF40', 'imec0.lf#LF41', 'imec0.lf#LF42', 'imec0.lf#LF43',
       'imec0.lf#LF44', 'imec0.lf#LF45', 'imec0.lf#LF46', 'imec0.lf#LF47',
       'imec0.lf#LF48', 'imec0.lf#LF49', 'imec0.lf#LF50', 'imec0.lf#LF51',
       'imec0.lf#LF52', 'imec0.lf#L

In [6]:
# Bird1 : LMAN 'safe' channels: 202-236; AreaX: 50-176 (Nchan slightly overestimating)
areaX_channels = rec_lf.channel_ids[50:190][::10] # to make multiples of 10
lman_channels = rec_lf.channel_ids[202:252][::10]  # to make multiples of 10
unit_related_channels = [
'imec0.lf#LF224'
] # got this from filter+cmr_radius/tridesclous/

selection_list = np.concatenate([areaX_channels, lman_channels, unit_related_channels])

In [7]:
selection_list = sorted(np.unique(selection_list))
print(selection_list)

['imec0.lf#LF100', 'imec0.lf#LF110', 'imec0.lf#LF120', 'imec0.lf#LF130', 'imec0.lf#LF140', 'imec0.lf#LF150', 'imec0.lf#LF160', 'imec0.lf#LF170', 'imec0.lf#LF180', 'imec0.lf#LF202', 'imec0.lf#LF212', 'imec0.lf#LF222', 'imec0.lf#LF224', 'imec0.lf#LF232', 'imec0.lf#LF242', 'imec0.lf#LF50', 'imec0.lf#LF60', 'imec0.lf#LF70', 'imec0.lf#LF80', 'imec0.lf#LF90']


In [8]:
sliced_channels = rec_lf.channel_slice(channel_ids=selection_list)
chunked_data = sliced_channels.frame_slice(18000*rec_lf.get_sampling_frequency(), 21600*rec_lf.get_sampling_frequency())
chunked_data

FrameSliceRecording: 20 channels - 1 segments - 2.5kHz - 3600.000s

In [9]:
traces_channels = sliced_channels.get_traces(return_scaled=True)
transposed_traces = traces_channels.T

In [10]:
# this should be true
len(traces_channels[:,0])/rec_lf.get_sampling_frequency() == rec_lf.get_total_duration()

True

In [11]:
transposed_traces

array([[103.125 ,  98.4375, 126.5625, ...,  65.625 ,  60.9375,  37.5   ],
       [154.6875, 173.4375, 201.5625, ...,   4.6875,  18.75  , -18.75  ],
       [ 89.0625,  93.75  , 121.875 , ..., -84.375 , -84.375 , -89.0625],
       ...,
       [ 51.5625,  65.625 ,  65.625 , ..., 103.125 ,  98.4375,  98.4375],
       [-56.25  , -37.5   , -18.75  , ...,  46.875 ,  42.1875,  23.4375],
       [ 18.75  ,  32.8125,  37.5   , ...,  42.1875,  23.4375,   4.6875]],
      dtype=float32)

In [12]:
# this should be True
sliced_channels.get_channel_ids() == selection_list

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [13]:
for trace_indx in range(len(selection_list)):
    name_channel = selection_list[trace_indx]
    print(name_channel)
    trace = transposed_traces[trace_indx]
    print(trace)
    name_channel = selection_list[trace_indx]
    np.save(output_folder / (name_channel + '.npy'), trace)

imec0.lf#LF100
[103.125   98.4375 126.5625 ...  65.625   60.9375  37.5   ]
imec0.lf#LF110
[154.6875 173.4375 201.5625 ...   4.6875  18.75   -18.75  ]
imec0.lf#LF120
[ 89.0625  93.75   121.875  ... -84.375  -84.375  -89.0625]
imec0.lf#LF130
[ -28.125    18.75     51.5625 ... -103.125   -84.375  -121.875 ]
imec0.lf#LF140
[ 14.0625  23.4375  42.1875 ... 112.5     89.0625  98.4375]
imec0.lf#LF150
[ 93.75   107.8125 140.625  ... 107.8125  98.4375  84.375 ]
imec0.lf#LF160
[182.8125 192.1875 215.625  ... 182.8125 178.125  145.3125]
imec0.lf#LF170
[168.75   164.0625 201.5625 ... 276.5625 267.1875 257.8125]
imec0.lf#LF180
[-37.5    -18.75    23.4375 ... 295.3125 295.3125 290.625 ]
imec0.lf#LF202
[-46.875  -51.5625 -32.8125 ... 356.25   332.8125 314.0625]
imec0.lf#LF212
[-243.75   -243.75   -225.     ...   23.4375    9.375     9.375 ]
imec0.lf#LF222
[ -84.375  -121.875  -107.8125 ...  262.5     243.75    262.5   ]
imec0.lf#LF224
[-23.4375 -28.125  -14.0625 ... 295.3125 276.5625 276.5625]
imec0.l

## In case you need to check probe

In [ ]:
rec_ap = si.SpikeGLXRecordingExtractor(folder, stream_id='imec0.ap') # microphone

In [ ]:
rec_lf.set_probe(rec_ap.get_probe(), in_place=True)

In [ ]:
def check_probe_channels(rec):
    probe = rec.get_probe()
    plot_probe(probe, show_channel_on_click=True)
    plt.show()

check_probe_channels(rec_lf)

In [ ]:
location = rec_lf.get_channel_locations()
plt.scatter(location[:,0], location[:,1])
plt.show()

## Get spiketrains

In [5]:
sorting_path = Path('/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_6_22_11_2021_g0/sorting_20220419/full/filter+cmr_radius/tridesclous/custom_tdc_1/')

In [6]:
sorting_tdc = si.TridesclousSortingExtractor(sorting_path)
print(sorting_tdc)

PermissionError: [Errno 13] Permission denied: '/home/admin/smb4k/NAS5802A5.LOCAL/Public/Neuropixel_Recordings/Imp_10_11_2021/Rec_6_22_11_2021_g0/sorting_20220419/full/filter+cmr_radius/tridesclous/custom_tdc_1/channel_group_0/segment_0/processed_signals.raw'

In [ ]:
unit_list = sorting_tdc.unit_ids

In [ ]:
for unit in unit_list:
    unit_spks = sorting_tdc.get_unit_spike_train(unit_id=unit, return_times=True)
    unit_name = 'unit_'+ str(unit)
    np.savetxt(output_folder/(unit_name+'.txt'), unit_spks)